In [3]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime 
import matplotlib.pyplot as plt 
import math 
import uuid

from utils.utils import setLinkEod, setLinkIntd, getBuyDay

In [6]:
contracts17 = pd.read_csv("data/portfolioTesting/c16.csv")
contracts17['Start Date']= pd.to_datetime(contracts17['Start Date'])
contracts17=contracts17.drop_duplicates(subset='Award ID').sort_values(by='Start Date')

In [17]:
from dataclasses import dataclass 
import uuid


# datastructure to track trades made on the same underlying via contractID 
@dataclass
class transaction: 
    contractID: int
    num: int

# globally unique id generator 
class uniqueId:
    def __init__(self):
        self.id = str(uuid.uuid4())

In [16]:
class BenchmarkPortfolio:
    """ a benchmarked portfolio that mimicks all trades from the strategy using the SP500 
        
        id: a unique 6 digit number to keep track of the portfolio when multithreading
        capital: current capital of the portfolio
        buys: a list of trades made on the SP500 
        valuation: tracks the value of the portfolio after each transaction 
        holds = flag to track if any pending assets need to be sold before next purcahse
        bankrupt: well... if the portfolio has sufficient funds to continue
    """
    def __init__(self, c):
        self.id = uniqueId().id
        self.capital = c
        self.trades = []
        self.valuation = [c]
        self.holds = False
        self.bankrupt = False
        
    def buy(self, pps, shares, id, buyDate):
        if self.capital >= pps*shares: 
            if self.holds == False:
                self.capital -= pps*shares
                self.trades.append(transaction(id, shares))
                self.valuation.append(self.capital)
                self.holds = True 
                print(f"B{shares}{'SPY'}{pps}D{buyDate}")

                return True 
            else: 
                print ("there are pending market positions to be cleared")
                return False 
        else: 
            print("insufficient capital to complete transaction")
            self.bankrupt = True   
            return False 

    def sell(self, pps, id, sellDate): 
        for transaction in reversed(self.trades):
            if transaction.contractID == id: 
                self.capital += transaction.num * pps 
                self.valuation.append(self.capital)
                print(f"S{transaction.num}{'SPY'}{pps}D{sellDate}")
                break 

    def sellSuccessful(self):
        self.holds = False


In [22]:
class Portfolio(BenchmarkPortfolio) : 
    """ a simulated portfolio that automatcally purchases and sells 
        assets using the government contracts + MR strategy 

        trades: a dictionary of trades made under (K, V) = (Ticker, numShares)
        mRThresh = mean reversion z-score treshold to sell
        weight = percentage of total cap to allocate to each trade 
    """
    def __init__(self, t, r, w, c):
        super.__init__(c)
        self.trades = {}
        self.mRThresh = t 
        self.weight = w

    def logTrade(self, ticker, id, shares): 
        if ticker not in dict: 
            self.trades[ticker] = [transaction(id, shares)]
        else: 
            self.trades[ticker].append(transaction(id, shares))
                        
    def buy(self, ticker, pps, shares, id, buyDate): 
        if self.capital >= pps*shares: 
            if self.holds == False: 
                self.capital -= pps*shares
                self.logTrade(self, self.trades, ticker, id, shares)
                self.buys.append([transaction(id, shares), buyDate])
                self.holds = True
                self.valueation.append(self.capital)
                print(f"B{shares}{ticker}{pps}D{buyDate}")
                return True
            else: 
                print ("there are pending market positions to be cleared")
                return False 

        else: 
            print("insufficient capital to complete transaction")
            self.bankrupt = True   
            return False 
    
    def sell(self, pps, iD, ticker, sellDate): 
        if ticker in self.trades: 
            for transaction in self.trades[ticker]: 
                if transaction.contractID == iD: 
                    shares = transaction.num 
                    self.capital += shares * pps
                    self.trades[ticker].remove(transaction) 
                    print(f"B{shares}{ticker}{pps}D{sellDate}")

In [1]:
exceptions = []

In [24]:
contracts17.iloc[0]

internal_id                                                            28086905
Award ID                                                                   SD14
Recipient Name                                                              NOC
Award Amount                                                         42197144.0
Total Outlays                                                               NaN
Description                   IGF::OT::IGF  CY16 PERFORMANCE-BASED LOGISTICS...
Contract Award Type                                              DELIVERY ORDER
def_codes                                                                   NaN
COVID-19 Obligations                                                        NaN
COVID-19 Outlays                                                            NaN
Infrastructure Obligations                                                  NaN
Infrastructure Outlays                                                      NaN
Awarding Agency                         

In [ ]:
def filterResponse(response, ticker, date):
    if response.status_code == 200: 
        if len(response.json()) == 0: 
            response = requests.get(url=setLinkEod(ticker, date, date))
            try: 
                pps = response.json()['historical']['close']
                return pps
            except Exception as e: 
                print ("Unexpected exception: " + str(e))
                exceptions.append(f"{str(e)} on buy order of {ticker} on {date} via contract number: {id}")
                return None
        elif len(response.json>15): 
            pps = response.json()[-15]["open"]
            return pps 

In [ ]:
def meanReversion(): 
    pass 

In [ ]:
def buyOrder(portfolio, benchPortfolio, w, ticker, date, id): 
    global exceptions 
    apiResponse = requests.get(url=setLinkIntd(ticker, date, date, "5min"))
    spyResponse = requests.get(url=setLinkIntd("^SPX", date, date, "5min"))
    
    pps = filterResponse(apiResponse, ticker, date)
    ppsSPX = filterResponse(spyResponse, "^SPX", date)

    if (pps!=None and ppsSPX!=None): 
        shares = math.floor((w*portfolio.capital)/pps)
        sharesSPX = math.floor((w*benchPortfolio.capital)/ppsSPX)
        portfolio.buy(ticker, pps, shares, id, date)
        benchPortfolio.buy(ppsSPX, sharesSPX, id, date)
        return True 
    else:
        return False

In [ ]:
def sellOrder(portfolio, benchPortfolio, w, ticker, date, id):
    